In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
raw_files_path = ['/content/drive/MyDrive/DSE_Project/rawData/digana_2022_07.csv',
             '/content/drive/MyDrive/DSE_Project/rawData/digana_2022_08.csv']

dataOutPath = '/content/drive/MyDrive/DSE_Project/dataOut'

tripEndsPath = '/content/drive/MyDrive/DSE_Project/dataOut/trip_ends.csv'
busTripsPath = '/content/drive/MyDrive/DSE_Project/dataOut/bus_trips.csv'
busStopsAndTerminalsPath = '/content/drive/MyDrive/DSE_Project/bus_stops_and_terminals_654.csv'


In [31]:
# -*- coding: utf-8 -*-
"""bus_stops_extraction.ipynb
Original file is located at
    https://colab.research.google.com/drive/1OikJekLXp81Irr14GkLemhqGEyvsEXlh
Importing python libraries
"""

import pandas as pd
import numpy as np
from datetime import datetime,date, timedelta

! pip install geopandas
import geopandas as gpd
from geopandas import GeoDataFrame as gdf


import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster





In [32]:
#Importing
# 1. raw GPS data
# 2. splitted trip data from bus_trip_extraction.py
# 3. bus stops details (Latitude and longitude)


trip_ends = pd.read_csv(tripEndsPath)
bus_trips = pd.read_csv(busTripsPath)
bus_stops= pd.read_csv(busStopsAndTerminalsPath)


In [33]:

# Create an empty list to store DataFrames
dfs = []

# Loop through the list of file names and read each CSV into a DataFrame
for file in raw_files_path:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate the DataFrames vertically
raw_data = pd.concat(dfs, ignore_index=True)

In [34]:
def raw_data_cleaning(raw_data):
    """
    Removal of records with error records.
    Remove data with zero values for longitude and latitude columns.
    Remove data with dates outside the desired range.
    Sort data by time and device.

    Args:
        raw_data (pd.DataFrame): Crude raw GPS data filtered out from the server for the required time window.

    Returns:
        gps_data (pd.DataFrame): A cleaned dataframe object of GPS data.
    """

    # raw_data = raw_data.drop(drop_columns, axis=1)

    gps_data = raw_data[raw_data.latitude != 0]
    gps_data = gps_data[gps_data.longitude != 0]  # cleaning zero values for latitude & longitude

    gps_data['date'] = pd.to_datetime(gps_data['devicetime']).dt.date  # split date and time separately into datetime variables
    gps_data['time'] = pd.to_datetime(gps_data['devicetime']).dt.time

    # Remove rows with dates outside the desired range
    start_date = pd.Timestamp("2021-10-01").date()
    end_date = pd.Timestamp("2022-10-31").date()
    gps_data = gps_data[(gps_data['date'] >= start_date) & (gps_data['date'] <= end_date)]

    gps_data = gps_data.sort_values(['deviceid', 'date', 'time'])  # sorting dataset by time and device

    return gps_data



In [35]:

additional_columns = ['servertime','fixtime','address','routeid']
try:
  # drop the additional columns
  raw_data = raw_data.drop(additional_columns, axis = 1)
except Exception as e:
  print(e)
  pass


gps_data= raw_data_cleaning(raw_data)

"['servertime', 'fixtime', 'address', 'routeid'] not found in axis"


In [36]:
bus_stops

,stop_id,route_id,direction,address,latitude,longitude
0,BT01,654,Kandy-Digana,Kandy,7.292462,80.634978
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030
6,106,654,Kandy-Digana,Kalapura Junction Busstop,7.279830,80.676210
7,107,654,Kandy-Digana,Nattarampotha Junction Bus Stop,7.279117,80.679450
8,108,654,Kandy-Digana,Kundasale New town,7.280900,80.684160
9,109,654,Kandy-Digana,Warapitiya,7.281490,80.686350


In [69]:
#developing geo-buffer rings around every bus stops
def bus_stop_buffer_create(gps_data,bus_stops,stop_buffer,extra_buffer):

  """

    Buffer and additional buffer  created  to accomodate points if they were missed in standard stop buffer.

    Args:
        gps_data (pd.DataFrame): Cleaned gps data filtered out from the server for the required time window.
        bus_stops (pd.DataFrame) : Bus stops data for the trip route
        stop_buffer (int):  Radius of the buffer area to represent bus stops
        extra_buffer (int):  Extended radius of the buffer area to represent bus stops.

    Returns:
        bus_stops_buffer1 (GeoDataFrame) : Buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2 (GeoDataFrame) : Buffer created for filtered  Digana-Kandy direction
        gps_data (GeoDataFrame) :  GPS data as GeoDataFrame with projected corrdinates.
        bus_stops_buffer1_add (GeoDataFrame) : Additional buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2_add (GeoDataFrame) : Additional buffer created for filtered  Digana-Kandy direction.
  """

  #Create Geodataframe of GPS data and bus stops data
  gps_data = gpd.GeoDataFrame(gps_data, geometry=gpd.points_from_xy(gps_data.longitude,gps_data.latitude),crs='EPSG:4326')
  bus_stops = gpd.GeoDataFrame(bus_stops, geometry=gpd.points_from_xy(bus_stops.longitude,bus_stops.latitude),crs='EPSG:4326')

  #project the corrdinates in Local coordinate system
  bus_stops = bus_stops.to_crs('EPSG:5234')
  gps_data = gps_data.to_crs('EPSG:5234')

  #split bus stops dataframe into two based on route direction
  bus_stops_direction1 = bus_stops[bus_stops['direction']=='Kandy-Digana']
  bus_stops_direction2 = bus_stops[bus_stops['direction']=='Digana-Kandy']

  bus_stops_direction1.reset_index(drop = True, inplace = True)
  bus_stops_direction2.reset_index(drop = True, inplace = True)

  #proximity analysis
  #creating a buffer
  bus_stops_buffer1 = gpd.GeoDataFrame(bus_stops_direction1, geometry = bus_stops_direction1.geometry.buffer(stop_buffer))
  bus_stops_buffer2 = gpd.GeoDataFrame(bus_stops_direction2, geometry = bus_stops_direction2.geometry.buffer(stop_buffer))

  #creating additional extra buffer to accomodate points if they were missed in standard stop buffer
  bus_stops_buffer1_add = gpd.GeoDataFrame(bus_stops_direction1, geometry = bus_stops_direction1.geometry.buffer(extra_buffer))
  bus_stops_buffer2_add = gpd.GeoDataFrame(bus_stops_direction2, geometry = bus_stops_direction2.geometry.buffer(extra_buffer))

  return bus_stops_buffer1, bus_stops_buffer2,gps_data,bus_stops_buffer1_add,bus_stops_buffer2_add

stop_buffer = 50
extra_buffer = 100
bus_stops_buffer1, bus_stops_buffer2,gps_data,bus_stops_buffer1_add,bus_stops_buffer2_add = bus_stop_buffer_create(gps_data,bus_stops,stop_buffer,extra_buffer)


In [70]:
#splitting trajectories
def bus_trajectory(gps_data,trip_ends,bus_trips):

  """
    Create bus trajectory data of sequence of bus stops with direction of trip.

    Args:
        gps_data (GeoDataFrame): Bus trips GPS data
        trip_ends (pd.DataFrame) : Splitted trip data from bus_trip_extraction.py
        bus_trips (pd.DataFrame) : Bus trips data

    Returns:
        bus_trajectory (pd.DataFrame): Sequence of bus trip trajectory data
  """

  #gps records that are matched with end terminals, are merged with whole GPS records
  trip_ends = trip_ends[['id','bus_stop','trip_id']]
  bus_trajectory = pd.merge(left = gps_data, right  = trip_ends,how = 'outer',left_on ='id', right_on= 'id')

  #gps records that are not associated with the terminals are asssigned as trip id = 0
  bus_trajectory["trip_id"].fillna(0, inplace = True)

  #run a loop to assign trip_id to records that are in between the terminals
  bus_trajectory.reset_index(drop = True, inplace = True)

  trip =1
  for i in range(len(bus_trajectory)-1):
    if (bus_trajectory.at[i,'trip_id']==trip) & (bus_trajectory.at[i+1, 'trip_id'] == 0):
      bus_trajectory.at[i+1,'trip_id'] = trip
    elif (bus_trajectory.at[i,'trip_id']==trip) & (bus_trajectory.at[i+1, 'trip_id'] == trip):
      trip = trip + 1

  bus_trajectory.drop(bus_trajectory[bus_trajectory['trip_id']==0].index, inplace = True ) #drop records that are not identified as a bus trip

  #Identify the directions of each bus trajectories using bus trips extracted data
  directions= bus_trips.set_index('trip_id').to_dict()['direction']
  bus_trajectory['direction'] = list(map(lambda x: directions[x]   ,bus_trajectory['trip_id']))

  return bus_trajectory

bus_trajectory = bus_trajectory(gps_data,trip_ends,bus_trips)

In [72]:
# @title Default title text
def stop_buffer_filter(bus_trajectory,bus_stops_buffer1,bus_stops_buffer2,bus_stops_buffer1_add,bus_stops_buffer2_add):

  """

    Filter bus trip data of two buffer ranges with all the bus points, only bus stops points.

    Args:
        bus_trajectory (pd.DataFrame): Sequence of bus trip trajectory data
        bus_stops_buffer1 (GeoDataFrame) : Buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2 (GeoDataFrame) : Buffer created for filtered  Digana-Kandy direction
        bus_stops_buffer1_add (GeoDataFrame) : Additional buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2_add (GeoDataFrame) : Additional buffer created for filtered  Digana-Kandy direction.

    Returns:
        bus_trip_all_points (pd.DataFrame): Bus trip data with all points including null for bus_stop
        bus_stop_all_points (pd.DataFrame): Bus trip data with only bus_stops points

  """

  #project to local coordinate system before buffer filtering
  bus_trajectory = bus_trajectory.to_crs('EPSG:5234')

  #split trajectories by direction
  trajectory_dir_1 = bus_trajectory[bus_trajectory['direction'] == 1]
  trajectory_dir_2 = bus_trajectory[bus_trajectory['direction'] == 2]

  #reset index before for loop
  trajectory_dir_1.reset_index(drop = True, inplace = True)
  trajectory_dir_2.reset_index(drop = True, inplace = True)


  #filter records within bus stops buffer of both directions
  for i in range(len(trajectory_dir_1)):
    for stop in range(len(bus_stops_buffer1)):
      if bus_stops_buffer1.iloc[stop].geometry.contains(trajectory_dir_1.iloc[i].geometry):
        trajectory_dir_1.at[i,'bus_stop'] = bus_stops_buffer1.at[stop,'stop_id']
      else:
        if bus_stops_buffer1_add.iloc[stop].geometry.contains(trajectory_dir_1.iloc[i].geometry):
          trajectory_dir_1.at[i,'bus_stop'] = bus_stops_buffer1_add.at[stop,'stop_id']

  #filter records within bus stops buffer of both directions
  for i in range(len(trajectory_dir_2)):
    for stop in range(len(bus_stops_buffer2)):
      if bus_stops_buffer2.iloc[stop].geometry.contains(trajectory_dir_2.iloc[i].geometry):
        trajectory_dir_2.at[i,'bus_stop'] = bus_stops_buffer2.at[stop,'stop_id']
      else:
        if bus_stops_buffer2_add.iloc[stop].geometry.contains(trajectory_dir_2.iloc[i].geometry):
          trajectory_dir_2.at[i,'bus_stop'] = bus_stops_buffer2_add.at[stop,'stop_id']

  #concatenate dataframes of both directions and keep only records filtered within bus stops
  bus_trip_all_points = pd.concat([trajectory_dir_1,trajectory_dir_2])
  bus_stop_all_points = bus_trip_all_points.dropna()

  return bus_trip_all_points,bus_stop_all_points

bus_trip_all_points,bus_stop_all_points = stop_buffer_filter(bus_trajectory,bus_stops_buffer1,bus_stops_buffer2,bus_stops_buffer1_add,bus_stops_buffer2_add)

In [76]:
# save bus_trip_all_points,bus_stop_all_points as csv files
bus_trip_all_points.to_csv(dataOutPath + '/bus_trip_all_points.csv',index=False)
bus_stop_all_points.to_csv(dataOutPath + '/bus_stop_all_points.csv',index=False)

In [ ]:
bus_trip_all_points

In [ ]:
bus_stop_all_points

In [75]:
def dwell_time_estimation(bus_stop_all_points):

  """
    Drop terminal points from all points data.
    Calculate arrival_time, departure_time according to check whether grouped record has 0 speed values or not.
    Dwell time derived in seconds.

    Args:
        bus_stop_all_points (pd.DataFrame): Bus trip data with only bus_stops points

    Returns:
        bus_stop_times (pd.DataFrame): Bus stops data with arrival_time, departure_time, dwell_time for each stops excluding terminals.

   """

  #Drop records with End Bus terminals
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT01'].index, inplace = True)
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT02'].index, inplace = True)

  #grouping all records filtered for every bus stop
  bus_stop_all_points['grouped_ends'] = ((bus_stop_all_points['bus_stop'].shift() != bus_stop_all_points['bus_stop'])).cumsum()

  #creating a new dataframe for bus stop times
  columns = ['trip_id','deviceid','date','direction','bus_stop', 'arrival_time','departure_time','dwell_time']
  bus_stop_times = pd.DataFrame(columns=columns)

  #Loop over every grouped filtered records and choose the two records that indicate bus arrival and departure to the stop
  for name, group in bus_stop_all_points.groupby('grouped_ends'):
    if 0 in group['speed'].values:               #if the grouped filter record has '0" speed values, then bus has stopped more than 15 seconds there and first '0'speed record as the arrival
      values = []
      trip_id = np.unique(group['trip_id'].values)[0]
      direction = np.unique(group['direction'].values)[0]
      deviceid = np.unique(group['deviceid'].values)[0]
      date = np.unique(group['date'].values)[0]
      bus_stop = np.unique(group['bus_stop'].values)[0]

      arrival_time = group[group['speed']==0]['time'].min()

      buffer_leaving_time = group['time'].max()
      rough_departure_time = group[group['speed']==0]['time'].max()

      if (datetime.combine(date.min,buffer_leaving_time) - datetime.combine(date.min,rough_departure_time)).total_seconds() > 15:
        departure_time = (datetime.combine(date.min,rough_departure_time) + timedelta(seconds =15)).time()
      else:
        departure_time = buffer_leaving_time

      values.extend([trip_id,deviceid,date,direction,bus_stop,arrival_time,departure_time])
      new_row = pd.DataFrame([dict(zip(columns, values))])
      bus_stop_times = pd.concat([bus_stop_times, new_row], ignore_index=True)


    else:
      values = []
      trip_id = np.unique(group[['trip_id']].values)[0]
      direction = np.unique(group['direction'].values)[0]
      deviceid = np.unique(group[['deviceid']].values)[0]
      date = np.unique(group['date'].values)[0]
      bus_stop = np.unique(group['bus_stop'].values)[0]

      arrival_time = group['time'].min()
      departure_time = arrival_time

      values.extend([trip_id,deviceid,date,direction,bus_stop,arrival_time,departure_time])
      new_row = pd.DataFrame([dict(zip(columns, values))])
      bus_stop_times = pd.concat([bus_stop_times, new_row], ignore_index=True)

  for i in range(len(bus_stop_times)):
    bus_stop_times.at[i,'dwell_time'] = datetime.combine(date.min,bus_stop_times.at[i,'departure_time']) - datetime.combine(date.min,bus_stop_times.at[i,'arrival_time'])

  bus_stop_times['dwell_time_in_seconds'] =  bus_stop_times['dwell_time']/np.timedelta64(1,'s')

  return bus_stop_times

bus_stop_times = dwell_time_estimation(bus_stop_all_points)

<ipython-input-75-f60cbf412698>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT01'].index, inplace = True)
<ipython-input-75-f60cbf412698>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT02'].index, inplace = True)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [77]:
def dwell_time_feature_addition(bus_stop_times):

  """
    To created aditonal derieved features for bus stops data.

    Args:
        bus_stop_times (pd.DataFrame): Bus stops data with arrival_time, departure_time, dwell_time for each stops excluding terminals.

    Returns:
        bus_stop_times (pd.DataFrame): Bus stops data with created features.

  """

  #bus_stop_times = bus_stop_times.drop(bus_stop_times[bus_stop_times['dwell_time_in_seconds']>threshold].index )

  bus_stop_times['day_of_week'] = pd.to_datetime(bus_stop_times['date']).dt.weekday
  bus_stop_times['hour_of_day'] = list(map(lambda x: x.hour, (bus_stop_times['arrival_time'])))
  bus_stop_times['weekday/end'] = list(map(lambda x: 1 if x < 5 else 0 , (bus_stop_times['day_of_week'])))

  return bus_stop_times

threshold = 480
bus_stop_times = dwell_time_feature_addition(bus_stop_times)
# save as a csv file
bus_stop_times.to_csv(dataOutPath + '/bus_stop_times.csv', index=False)

In [ ]:
bus_stop_times

In [ ]:
bus_stops

In [ ]:
bus_stops_dir_1= bus_stops[bus_stops['direction']=='Kandy-Digana']

In [ ]:
bus_stops_dir_1

In [ ]:
# from shapely.geometry import Point

# def trip_visualization(trip_id, city_location,bus_stops_dir_1):
#     """
#     Using a  GPS data visualization package of Folium, project the coordinates on
#     Open Street Map (OSM) to explore how the records are spread and to gain some insights and overview.

#     Args:
#         trip_id (int):  A random trip id to visualize its coordinates' distribution
#         city_location (arr): Longtitude and latitude of city.
#         bus_stop_dir_1 (GeoDataFrame):

#     Returns:
#         map (MapObject):  A visualizable Map Object for selected trip id .
#         bus_stops_map () :

#     """
#     geometry = [Point(xy) for xy in zip(bus_stops_dir_1['longitude'], bus_stops_dir_1['latitude'])]
#     bus_stops_dir_1 = gpd.GeoDataFrame(bus_stops_dir_1, geometry=geometry)
#     trip = bus_trajectory[bus_trajectory['trip_id']==trip_id]
#     trip = trip.to_crs('EPSG:4326')
#     bus_stops_dir_1 = bus_stops_dir_1.to_crs('EPSG:4326')

#     map = folium.Map(location=city_location, tiles='openstreetmap', zoom_start=14)
#     for idx, row in trip.iterrows():
#         Marker([row['latitude'], row['longitude']]).add_to(map)

#     bus_stops_map = folium.Map(location=city_location, tiles='openstreetmap', zoom_start=14)
#     for idx, row in bus_stop_dir_1.iterrows():
#         Marker([row['latitude'], row['longitude']]).add_to(bus_stops_map)

#     return map, bus_stops_map

# trip_id = 1560  #Choose a random trip id to visualize its coordinates' distribution
# city_location = [7.2906,80.6337] #kandy city location

# map, bus_stops_map = trip_visualization(trip_id,city_location,bus_stops_dir_1)
